# Infobae

En esta notebook veremos qué se puede hacer con los artículos de infobae...

In [1]:
%load_ext autoreload
%autoreload 2
import json
from mongoengine import connect
from tweepyrate import create_apps
from hatespeech_models import Tweet, Article

client = connect("hatespeech-news")
db = client["hatespeech-news"]


with open("../../dumps/coronavirus-argentina-v3.json") as f:
    arts = json.load(f)

In [2]:
from tqdm.auto import tqdm
import datetime

newspaper = "infobae"
newspaper_articles = []

for art in tqdm(arts):
    if art["user"] == newspaper:
        date = art["created_at"]["$date"]
        art["created_at"] = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ")
        newspaper_articles.append(art)
    

In [3]:
newspaper_articles = sorted(newspaper_articles, key=lambda l: l["created_at"], reverse=True)

newspaper_articles[0]["created_at"]



for art in newspaper_articles[:30]:
    print("="*80 +'\n\n')
    print(art["created_at"], '\n\n')
    print(art["title"], "\n\n")
    print(art["url"], "\n\n")
    print(":"*30, '\n\n')
    print(art["body"])



2020-09-22 15:53:10 


Sin anuncio oficial sobre el regreso de los vuelos, las aerolíneas postergaron su cronograma para el 12 de octubre 


https://www.infobae.com/economia/2020/09/22/sin-anuncio-oficial-sobre-el-regreso-de-los-vuelos-las-aerolineas-postergaron-su-cronogramas-para-el-12-de-octubre/?utm_medium=Echobox&utm_source=Twitter#Echobox=1600789427 


:::::::::::::::::::::::::::::: 


\n

Durante varias semanas, desde los ministerios de Turismo y de Transporte apostaron al 1º de octubre como la fecha para el reinicio de los vuelos regulares. Sin embargo, no hubo anuncios oficiales hasta ahora y las compañías aéreas ya comenzaron a reprogramar sus cronogramas de vuelos para el 12 de octubre, el día en que finaliza la última extensión del decreto presidencial que canceló las operaciones aéreas domésticas por la cuarentena.

Esta semana, las aerolíneas que operan en el mercado argentino recibieron una comunicación a través de un sistema interno de avisos —conocido como Notam (Not

Ok! Tenemos un problema a partir del ... 20 de Junio?

In [37]:
from hate_collector.article import body_getters

if article.doc.cssselect("pija"):
    1
else:
    print(-1)

-1


In [34]:
from hate_collector.article import download_article

download_article(missing_articles[0])

In [ ]:
for art in articles.limit(30):
    print("="*80 +'\n\n')
    print(art.created_at, '\n\n')
    print(art.title, "\n\n")
    print(f"comentarios: {len(art['comments'])}")
    print(art.url, "\n\n")
    print(":"*30, '\n\n')
    print(art.body)

In [55]:
from newspaper import Article as NSArticle
url = "https://www.infobae.com/politica/2020/07/31/jose-maria-campagnoli-que-este-beraldi-en-la-comision-es-un-mensaje-para-los-jueces-que-juzgan-a-cristina-kirchner/"
article = NSArticle(url, language="es")

article.download()
article.parse()

In [53]:
print(article.text)

El fiscal José María Campagnoli (NA)

El fiscal de distrito de Saavedra, José María Campagnoli, está en contra de la reforma judicial que esta semana presentó el gobierno de Alberto Fernández. “ Es un despilfarro de dinero en este contexto de miseria porque estamos en la peor crisis económica del país ”, dijo en diálogo con Infobae. Recordó el proyecto que había elaborado en los primeros años del gobierno de Néstor Kirchner cuando era funcionario del Ministerio de Justicia cuando al frente estaba Gustavo Béliz.

“Iban a haber unos 80 juzgados de primera instancia del crimen organizado y se ampliaba la competencia que llegaba hasta la ciudad de Rosario y el conurbano bonaerense, que es el corredor donde se cometen la mayor cantidad de delitos organizados. Eso era a costo cero no esta elefantística reforma que tiene previsto el nombramiento de más de 200 jueces”, recordó sobre esa iniciativa.

También opinó que la Corte debe tener cinco integrantes, que no se debe dividir en salas y que 

In [54]:
elem = article.doc.cssselect("article")[0]

children = list(elem.getchildren())


text_children = [c for c in children if c.tag == "p"]

text = "\n\n".join(t.text_content().strip() for t in text_children)

print(text)

El fiscal de distrito de Saavedra, José María Campagnoli, está en contra de la reforma judicial que esta semana presentó el gobierno de Alberto Fernández. “Es un despilfarro de dinero en este contexto de miseria porque estamos en la peor crisis económica del país”, dijo en diálogo con Infobae. Recordó el proyecto que había elaborado en los primeros años del gobierno de Néstor Kirchner cuando era funcionario del Ministerio de Justicia cuando al frente estaba Gustavo Béliz.

“Iban a haber unos 80 juzgados de primera instancia del crimen organizado y se ampliaba la competencia que llegaba hasta la ciudad de Rosario y el conurbano bonaerense, que es el corredor donde se cometen la mayor cantidad de delitos organizados. Eso era a costo cero no esta elefantística reforma que tiene previsto el nombramiento de más de 200 jueces”, recordó sobre esa iniciativa.

También opinó que la Corte debe tener cinco integrantes, que no se debe dividir en salas y que sí se le deben estipular plazos para res

In [38]:
"""
NOTICIAS VIEJAS
"""
elem = article.doc.get_element_by_id("article-content")

"""
Esto es código particular de infobae
Saco los links que están al pie de la nota
"""
children = list(elem.getchildren())

last = children[-1]

while len(list(children[-1].iter("a"))) > 0:
    children.pop(-1)
    
"""
Me quedo con el texto
"""
text_children = [t for t in children if t.tag not in ["meta", "script"]]
text = "\n\n".join(t.text_content().strip() for t in text_children)

print(text.strip())

[autoreload of hate_collector.article failed: Traceback (most recent call last):
  File "/home/jmperez/.local/share/virtualenvs/hatespeech-collection-qaK7Gomm/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jmperez/.local/share/virtualenvs/hatespeech-collection-qaK7Gomm/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jmperez/.local/share/virtualenvs/hatespeech-collection-qaK7Gomm/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/jmperez/.local/share/virtualenvs/hatespeech-collection-qaK7Gomm/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, 

KeyError: 'article-content'

## Noticias repetidas